In [3]:
using RCall

INFO: Precompiling module RCall.
Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{UInt8}, ::UInt32) at .\deprecated.jl:57
 [3] querykey(::UInt32, ::String) at C:\Users\jc3246\.julia\v0.6\WinReg\src\WinReg.jl:78
 [4] querykey(::UInt32, ::String, ::String) at C:\Users\jc3246\.julia\v0.6\WinReg\src\WinReg.jl:113
 [5] locate_Rhome_libR() at C:\Users\jc3246\.julia\v0.6\RCall\src\setup.jl:36
 [6] include_from_node1(::String) at .\loading.jl:569
 [7] include(::String) at .\sysimg.jl:14
 [8] include_from_node1(::String) at .\loading.jl:569
 [9] include(::String) at .\sysimg.jl:14
 [10] anonymous at .\<missing>:2
 [11] eval(::Module, ::Any) at .\boot.jl:235
 [12] process_options(::Base.JLOptions) at .\client.jl:286
 [13] _start() at .\client.jl:371
while loading C:\Users\jc3246\.julia\v0.6\RCall\src\setup.jl, in expression starting on line 51
INFO: Using R installation at C:\Program Files\R\R-3.4.0


In [2]:
Pkg.add("RCall")

INFO: Cloning cache of AxisArrays from https://github.com/JuliaArrays/AxisArrays.jl.git
INFO: Cloning cache of CategoricalArrays from https://github.com/JuliaData/CategoricalArrays.jl.git
INFO: Cloning cache of Combinatorics from https://github.com/JuliaMath/Combinatorics.jl.git
INFO: Cloning cache of DataArrays from https://github.com/JuliaStats/DataArrays.jl.git
INFO: Cloning cache of DataFrames from https://github.com/JuliaStats/DataFrames.jl.git
INFO: Cloning cache of DataStructures from https://github.com/JuliaCollections/DataStructures.jl.git
INFO: Cloning cache of FileIO from https://github.com/JuliaIO/FileIO.jl.git
INFO: Cloning cache of GZip from https://github.com/JuliaIO/GZip.jl.git
INFO: Cloning cache of IntervalSets from https://github.com/JuliaMath/IntervalSets.jl.git
INFO: Cloning cache of IterTools from https://github.com/JuliaCollections/IterTools.jl.git
INFO: Cloning cache of NamedArrays from https://github.com/davidavdav/NamedArrays.jl.git
INFO: Cloning cache of Null

In [5]:
R"library(raster)"
R"library(maptools)"
R"library(maps)"

 	Note: when rgeos is not available, polygon geometry 	computations in maptools depend on gpclib,
 	which has a restricted licence. It is disabled by default;
 	to enable gpclib, type gpclibPermit()


RCall.RObject{RCall.StrSxp}
 [1] "maps"      "maptools"  "raster"    "sp"        "stats"     "graphics" 
 [7] "grDevices" "utils"     "datasets"  "methods"   "base"     


In [6]:
cd("C:/Users/jc3246/Desktop/Soko_Lei/GridtoCounty")

In [7]:
R"urban1<-readShapePoly('urban1.shp')"
R"y<-raster(ncols=587,nrows=256,xmn=-125.1,xmx=-66.5,ymn=24.2,ymx=49.7)"
R"urbangrid1<-rasterize(urban1, y, 'URBANAP020') "
R"urbangrid1[is.na(urbangrid1)==TRUE]<-0"


RCall.RObject{RCall.RealSxp}
[1] 0


In [8]:
R"write.table(as.matrix(urbangrid1),'urbangrid1.txt',row.names=FALSE, col.names=FALSE)"

RCall.RObject{RCall.NilSxp}
NULL


In [9]:
urban=readdlm("urbangrid1.txt",header=false);

In [10]:
is_urban=urban;
for m=1:256
    for n=1:587
    if urban[m,n]>0
            is_urban[m,n]=1
        else
            is_urban[m,n]=0
        end
    end
end

In [11]:
R=3;V=1;
value1=sqrt(2)*V*[(1:R)' (R:-1:1)']/R;
value2=(ones(2*R,1)*value1).*(ones(2*R,1)*value1)';
raw=conv2(value2,is_urban);
raw[find(raw.<0.1)]=0;
urban_value=raw[R:(256+R-1),R:(587+R-1)];

In [23]:
writedlm("urban_value.txt", "urban_value")
    

In [50]:

R"urbanvalue<-as.matrix(read.table('urban_value.txt',header=F, sep = '\t',as.is=TRUE));"
R"rast1<-raster(urbanvalue)"
R"extent(rast1)=c(-125.1,-66.5,24.2,49.7)"
R"projection(rast1) = CRS('+proj=longlat +datum=WGS84')"
R"urbanvalue<-rast1"


RCall.RObject{RCall.S4Sxp}
class       : RasterLayer 
dimensions  : 256, 587, 150272  (nrow, ncol, ncell)
resolution  : 0.09982964, 0.09960938  (x, y)
extent      : -125.1, -66.5, 24.2, 49.7  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0 
data source : in memory
names       : layer 
values      : 0, 23.33333  (min, max)



In [51]:
R"urbanpoint<-rasterToPoints(urbanvalue, fun=NULL, spatial=FALSE);"
R"x<-urbanpoint[,1]";
R"y<-urbanpoint[,2]";
R"xy<-cbind(x,y)";
R"urban_value<-urbanpoint[,3]";

In [52]:
R"CountyBoundary<-readShapePoly('US_county_2000-simple-latlon.shp');"

RCall.RObject{RCall.S4Sxp}
class       : SpatialPolygonsDataFrame 
features    : 3141 
extent      : -179.1473, 179.7785, 18.91393, 71.38961  (xmin, xmax, ymin, ymax)
coord. ref. : NA 
variables   : 20
names       : DECADE,  NHGISNAM, NHGISST, NHGISCTY, ICPSRST, ICPSRCTY,  ICPSRNAM, STATENAM, ICPSRSTI, ICPSRCTYI, ICPSRFIP, STATE, COUNTY,  PID,    X_CENTROID, ... 
min values  :   2000, Abbeville,     010,     0010,       1,       10, ABBEVILLE,  Alabama,        0,         0,        0,   010,   0010,    0, -1.003157e+06, ... 
max values  :   2000,   Ziebach,     560,     8400,      82,      990,   ZIEBACH,  Wyoming,       82,      8400,        0,   560,   8400, 3109,  9.994701e+05, ... 


In [ ]:
R"county <-matrix(numeric(0), 3109,0);"
R"for(i in 1:3109){
  countyboundary <- subset(CountyBoundary,PID==i)
  temp_JJJ <- raster(res=10) 
  extent(temp_JJJ) <- extent(countyboundary)
  out <- rasterize(countyboundary, temp_JJJ,'PID')
  countyresult<- rasterize(xy,out, urban_value)
  county[i]<-cellStats(countyresult,sum)
}"